# Projeto 1 - Ciência dos Dados

Nome: João Gabriel Valentim Rocha

Nome: Enzo Dadier Lacks Zamberlan

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

In [1]:
#!pip install seaborn
#!pip install emoji
#!pip install pysinonimos
#!pip install nltk
#!pip install sklearn
#!pip install unidecode

___
Carregando algumas bibliotecas:

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import random

In [3]:
import re
import string
import emoji
import nltk
import pysinonimos.sinonimos as sinom
from emoji import UNICODE_EMOJI
import unidecode

In [4]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\gabri\Desktop\Matérias 2 semestre\cdados\P1\Classificador_automatico_de_sentimentos


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [5]:
filename = 'assets/loki.xlsx'

In [6]:
train = pd.read_excel(filename)
train

,Treinamento,Relevancia
0,mood de hj: bolsominion veio falar com loki:\n...,0
1,sdds do loki,1
2,"considerações finais de loki: parabéns marvel,...",1
3,@_namizinhaa @yaluv_ @_girlrainbow versão femi...,0
4,o loki merece todo o amor do mundo ? \n \n si...,1
...,...,...
295,o loki tbm ? percebi que nao sei mais nada sob...,1
296,tô desde as 9 fazendo atividade e o loki destr...,0
297,@torystyliinson @mobiusdaavt pede pra ele te m...,0
298,"bolsonaro tá desesperado. discurso inflado, go...",0


In [7]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test

,Teste,Relevancia
0,comecei loki e logo no primeiro episódio ele s...,1
1,@thylaspirk exatamente \nnão dá cara loki mere...,1
2,pq a marvel falou que loki é irmão se thor sen...,0
3,vou fazer com o loki 🥰🤣 https://t.co/qry6pytktj,1
4,@stawocon loki eu ainda vi dnv ksksksksksksksk...,1
...,...,...
195,"vou fazer uma xícara temática pra mim, mas tô ...",0
196,o primeiro dente de leite de loki caiu https:/...,0
197,@mobiusdaavt @loki_l4ufeys0n eu vou te bloquear,0
198,queria ter juntado uma semana de merda de loki...,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O classificador automático de sentimentos montado tem como critério utilizado a relevância de tweets que remetam a alguma reação ou laço sentimental em relação à série, sendo os mesmos positivos ou não (elogiando algum episódio, comentando valores da série, ou ainda, criticando os mesmos). Neste sentido, postagens que dizem respeito unicamente a aspectos nesse tocante, elogiando ou comentando sobre os atores da obra, como também tweets com vagas menções sobre a série, foram considerados como irrelevantes.

### Procedimentos e configurações iniciais

Para continuarmos, é necessário definir algumas funções básicas que vão nos ajudar:

1) Primeiro, vamos fazer a limpeza dos tweets. Retirando simbolos e pontuações que não serão contabilizadas para nossa análise

2)

In [8]:
# Passo 0
def separa_emoji(tweet):
    # Essa função separa os emojis e transcreve suas respectivas descrições
    modified=' '.join(emoji.get_emoji_regexp().split(tweet))
    modified=modified.split()
    for i,emoji1 in enumerate(modified):
        if emoji1 in UNICODE_EMOJI['pt']:
            modified[i]=UNICODE_EMOJI['pt'][emoji1].replace(':','')
        elif emoji1 in UNICODE_EMOJI['en']:
            modified[i]=UNICODE_EMOJI['en'][emoji1].replace(':','')
        else:
            continue
    modified=' '.join(modified)
        
    return modified

In [9]:
# Passo 1
def limpa_frase(frase):
    # Primeiro, poe todas as palavras com letras minúsculas
    aux = frase.lower()
    
    # Segundo, remove # and @
    aux = re.sub("@[A-Za-z0-9_]+","", aux)
    aux = re.sub("#[A-Za-z0-9_]+","", aux)
    
    # Terceiro, remove links
    aux = re.sub(r"http\S+", "", aux)
    aux = re.sub(r"www.\S+", "", aux)
    
    # Quarto, remove pontuação
    aux = re.sub('[()!?]', ' ', aux)
    aux = re.sub('\[.*?\]',' ', aux)
    
    # Quinto, separa e troca os emojis pela sua respectiva descrição
    aux = separa_emoji(aux)
    aux = aux.replace("_", "")
    
    # Sexto, remove acentos
    aux = unidecode.unidecode(aux)
    
    # Sétimo, remove não alfa-numericos
    aux = re.sub("[^a-z0-9]"," ", aux)
    
    return aux

In [10]:
# Passo 2
# transforma uma string em uma lista, de tal forma que é possível acessar palavra por palavra
def tokenize(frase):
    return frase.split()

In [11]:
# Passo 3
# Aqui serão contabilizadas as "stop words"
nltk.download('stopwords')
prep = nltk.corpus.stopwords.words('portuguese')
prep.append('')
prep = prep + ['n', 's']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
# Passo 4
# Essa função retira as stop words de uma lista tokenizada
def no_stop_words(token):
    clear = []
    for element in token:
        if element not in prep:
            clear.append(element)
    return clear

In [13]:
# Auxiliar para transformar de lista para string
def list_to_string(lista):
    return ' '.join(lista)

In [14]:
# Função final que limpa todo tweet e devolve em formato de string
def limpa_tweet(tweet):
    return list_to_string(no_stop_words(tokenize(limpa_frase(tweet))))

In [15]:
# Gera uma pd.Series do set_frases
def frases_to_series(set_frases):
    set_frases += ' '
    return pd.Series(tokenize(set_frases.sum()))

In [16]:
'''aux = train['Clean']
print(len(frases_to_series(aux)), len(frases_to_series_2(aux)))'''
#frases_to_series_2(aux)

"aux = train['Clean']\nprint(len(frases_to_series(aux)), len(frases_to_series_2(aux)))"

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [17]:
# Transformando palavras em variáveis categóricas
train['Treinamento'] = train['Treinamento'].astype('category')
test['Teste'] = test['Teste'].astype('category')

In [18]:
# Aplicando funções de limpeza e certificação no dataframe de treinamento da base de dados
train['Clean'] = train['Treinamento'].apply(limpa_tweet)
test['Clean'] = test['Teste'].apply(limpa_tweet)

In [19]:
# Separa os tweets relevantes dos irrelevantes
train_rel = train[train['Relevancia'] == 1]
train_irrel= train[train['Relevancia'] == 0]

In [20]:
# Gera uma pd.Series das palavras relevantes e irrelevantes do training set
palavras_rel = frases_to_series(train_rel['Clean'])
palavras_irrel = frases_to_series(train_irrel['Clean'])

In [21]:
# Gera duas listas com as palavras relevantes e irrelevantes e gera uma lista total
lista_palavras_rel = list(palavras_rel)
lista_palavras_irrel = list(palavras_irrel)
lista_palavras = lista_palavras_rel + lista_palavras_irrel
lista_palavras_sem_repeticao = list(set(lista_palavras))

In [22]:
# Gera um pd.Series da lista com todas as palavras
palavras = pd.Series(lista_palavras)

In [23]:
# Frequencias relativas
freq_palavras_relevantes = palavras_rel.value_counts(True)
freq_palavras_irrelevantes = palavras_irrel.value_counts(True)
freq_palavras_total = palavras.value_counts(True)

# Frequencias absolutas
freq_palavras_relevantes_abs = palavras_rel.value_counts()
freq_palavras_irrelevantes_abs = palavras_irrel.value_counts()
freq_palavras_total_abs = palavras.value_counts()



## Probabilidades 

Aqui abordaremos os valores das probabilidades que serão imprescindíveis para a construção do modelo. Em primeiro lugar, note que as probabilidade de serem relevantes ou serem irrelevantes podem ser expressas da seguinte forma:
 
 * $P(R) \rightarrow $  Probabilidade de ser relevante
 * $P(I)=P(R^c) \rightarrow $  Probabilidade de ser irrelevante
 
Além disso, por serem complementares, temos que: $P(R) + P(I) = P(R) + P(R^c) = 1$
 

In [24]:
# Utilizando a razão entre a quantidade de palavras da lista de relevantes pelo número total, temos P_R
P_R = len(lista_palavras_rel) / len(lista_palavras)

# Por complementar, temos P_Rc
P_Rc = 1 - P_R

## Probabilidades Condicionais e o Teorema de Bayes
Temos que, para o cálculo de probabilidades condicionais, fazemos uso da seguinte relação:

$$P(A|B) = \frac{P(A \cap B)}{P(B)}$$

Onde, 
* $P(A|B) \rightarrow $ Probabilidade de A ocorrer, dado que B ocorreu  
* $P(A \cap B) \rightarrow $ Probabilidade de A ocorrer e B ocorrer
* $P(B) \rightarrow $ Probabilidade de B ocorrer

Analogamente, temos que:
$$P(B|A) = \frac{P(B \cap A)}{P(A)}$$

Onde, 
* $P(B|A) \rightarrow $ Probabilidade de A ocorrer, dado que B ocorreu  
* $P(B \cap A) \rightarrow $ Probabilidade de B ocorrer e A ocorrer
* $P(A) \rightarrow $ Probabilidade de B ocorrer

Note que as probabilidades da interseção podem comutar os eventos A e B, portanto: $P(B \cap A) = P(A \cap B)$

Dessa forma, podemos extrair o conhecido Teorema de Bayes:

$$P(A|B) = P(B|A).\frac{P(A)}{P(B)}$$



## O Classificador de Naive Bayes

O Classificador de Naive Bayes se baseia na construção de um modelo bag-of-word. Na análise de sentimento, queremos responder a seguinte pergunta: "Qual a probabilidade dessa frase ser relevante, dado esse conjunto de palavras?". Nesse sentido, é necessário computar esse cálculo utilizando probabilidades condicionais e o resultado do teorema de Bayes encontrado na sessão anterior:

$$P(R|frase) = \frac{P(frase|R).P(R)}{P(frase)}$$

Dessa forma, nós vamos usar essa relação para encontrar $P(R|frase)$, ou seja, a probabilidade de uma frase ser relevante, dado o conjunto de palavras. Para tanto, tome como exemplo a frase: "meu avô ama isso", em termos de notação, queremos encontrar:

$$P(R|"meu avo ama isso") = \frac{P("meu avo ama isso"|R).P(R)}{P("meu avo ama isso")}$$

Para prosseguir, utilizaremos um processo de "Tokenização", que consiste em dividir a frase em pedaços menores (as palavras) e assumir que uma palavra não influencia na colocação da outra. Sabemos que isso não é verdade, mas utilizaremos por questões de simplificação (essa é a ingenuidade do classificador de Naive Bayes). Portanto, nosso cálculo ficara da seguinte forma:

$$P("meuavoamaisso"|R) = P("meu"|R).P("avô"|R).P("ama"|R).P("isso"|R)$$

Logo, nossa expressão final será:

$$P(R|"meu avo ama isso") = \frac{P("meu"|R).P("avô"|R).P("ama"|R).P("isso"|R).P(R)}{P("meu avo ama isso")}$$

Analogamente, para encontrarmos a probabilidade dele ser irrelevante, podemos fazer o mesmo cálculo:

$$P(I|"meu avo ama isso") = \frac{P("meu"|I).P("avô"|I).P("ama"|I).P("isso"|I).P(I)}{P("meu avo ama isso")}$$

Agora, basta compararmos os valores das probabilidades:

Se, $P(R|"meu avo ama isso") > P(I|"meu avo ama isso")$, então, é mais provável que a frase seja $relevante$


Caso contrário, $P(R|"meu avo ama isso") < P(I|"meu avo ama isso")$, então, é mais provável que a frase seja $irrelevante$

### Suavização de Laplace

É importante ressaltar que para o caso de uma determina palavra em uma frase não pertencer ao nosso conjunto universo, a probabilidade atribuida a essa palavra será zero. Porém, não podemos utilizar a probabilidade de 0 por razões algébricas, portanto, utilizaremos a <b><em>Suavização de Laplace</em></b>. Basicamente essa ferramenta irá nos ajudar a inserir a palavra "estranha" no quesito probabilístico das categorias discutidas.

$$P(palavra|W) = \frac{F_{AW}+1}{P_{W}+P_{W^c}}$$

Onde: 

$ F_{AW} \rightarrow$ Frequência absoluta da palavra na categoria W
    
$P_{W} \rightarrow$ Todas as palavras pertencentes às frases rotuladas como da categoria W
    
$P_{W^c} \rightarrow$ Todas as palavras pertencentes às frases rotuladas como da categoria W^c

Nesse sentido, tome $W$ como sendo o evento $R$ e $W^c$ como sendo o evento $I$ ou $R^C$. Dessa forma, a soma do valor 1 faz com que o nosso resultado de $P(R|frase)$ ou $P(I|frase)$ nunca se torne zero, mesmo que a frequência absoluta da palavra seja zero.


In [47]:
'''
A suavização de Laplace será utilizada para os casos em que uma determinada palavra da frase não se encontra 
no conjunto universo.
'''
def suavizacao_de_laplace(palavra, relevancia, freq_palavras_abs):
    try:
        FAW = freq_palavras_abs[palavra]
    except:
        FAW = 0
    if relevancia == 'R':
        return (FAW + 1) / (len(lista_palavras_rel) + len(lista_palavras_sem_repeticao))
    elif relevancia == 'I' or relevancia == 'Rc':
        return (FAW + 1) / (len(lista_palavras_irrel) + len(lista_palavras_sem_repeticao))

In [48]:
# Função que calcula a probabilidade que queremos para fazer a desigualdade e decidir a categoria mais provável
def P(relevancia, frase):
    if type(frase) != list:
        frase = tokenize(frase)
    if relevancia == 'R':
        P_F_dado_R = 1
        for palavra in frase:
            P_F_dado_R *= suavizacao_de_laplace(palavra, relevancia, freq_palavras_relevantes_abs)
        P_R_dado_F = P_F_dado_R * P_R
        return P_R_dado_F
            
    elif relevancia == 'I' or relevancia == 'Rc':
        P_F_dado_Rc = 1
        for palavra in frase:
            P_F_dado_Rc *= suavizacao_de_laplace(palavra, relevancia, freq_palavras_irrelevantes_abs)
        P_Rc_dado_F = P_F_dado_Rc * P_Rc
        return P_Rc_dado_F
    return None

In [49]:
# Função final que recorre ao modelo de Naive Bayes
def NaiveBayesModel(frase):
    if P('R', frase) > P('I', frase):
        return 1
    return 0

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [53]:
aux = test.copy()
aux['Modelo'] = aux['Clean'].apply(NaiveBayesModel)

In [54]:
aux.Modelo.value_counts()

0    111
1     89
Name: Modelo, dtype: int64

In [55]:
aux.Relevancia.value_counts()

0    132
1     68
Name: Relevancia, dtype: int64

In [38]:
train

,Treinamento,Relevancia,Clean,Modelo
0,mood de hj: bolsominion veio falar com loki:\n...,0,mood hj bolsominion veio falar loki nao loki b...,0
1,sdds do loki,1,sdds loki,1
2,"considerações finais de loki: parabéns marvel,...",1,consideracoes finais loki parabens marvel chor...,1
3,@_namizinhaa @yaluv_ @_girlrainbow versão femi...,0,versao feminina loki multiverso,0
4,o loki merece todo o amor do mundo ? \n \n si...,1,loki merece todo amor mundo sim sim sim,1
...,...,...,...,...
295,o loki tbm ? percebi que nao sei mais nada sob...,1,loki tbm percebi nao sei nada sobre mcu,0
296,tô desde as 9 fazendo atividade e o loki destr...,0,to desde 9 fazendo atividade loki destruiu gen...,0
297,@torystyliinson @mobiusdaavt pede pra ele te m...,0,pede pra mandar pix,0
298,"bolsonaro tá desesperado. discurso inflado, go...",0,bolsonaro ta desesperado discurso inflado golp...,0


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**